In [ ]:

import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold,train_test_split,cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc

import matplotlib.pyplot as plt

import json
import itertools
import math
import csv
import time
import pickle


In [ ]:

!pip install -U -q PyDrive 
  
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 
  
  
# Authenticate and create the PyDrive client. 
auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

## Read Dataset

In [ ]:
link=['https://drive.google.com/file/d/1H-DfAD-IBPkiygk2yb55jlXUFnd4RGnI/view?usp=sharing','https://drive.google.com/file/d/169SMiw2HvvsEpoFxJ9T-fA3dzGtM-iYF/view?usp=sharing','https://drive.google.com/file/d/1UdbiH188nsXmL60C4VLpv6o5FU8cgXnk/view?usp=sharing','https://drive.google.com/file/d/1hL42SicgeWWdk_lx9gkIZYJjVSfncSCe/view?usp=sharing']
names=['ransom_dataset_test25samples.csv','benign_dataset_test25samples.csv','ransom_dataset_train75samples.csv','benign_dataset_train75samples.csv']

for i in range(4):
  id=link[i].split("/")[-2]
  downloaded=drive.CreateFile({'id':id})
  downloaded.GetContentFile(names[i])



In [ ]:
ransom_train = pd.read_csv(names[2], nrows=75)
ransom_test = pd.read_csv(names[0], nrows=25)

benign_train = pd.read_csv(names[3], nrows=75)
benign_test = pd.read_csv(names[1], nrows=25)

In [ ]:
ransom_train = ransom_train[ransom_train.columns[0:75000]]
ransom_test = ransom_test[ransom_test.columns[0:75000]]
benign_train = benign_train[benign_train.columns[0:75000]]
benign_test = benign_test[benign_test.columns[0:75000]]


In [ ]:
ransom_train['class']=1
ransom_test['class']=1
benign_train['class']=0
benign_test['class']=0

In [ ]:
train_df = pd.concat([ransom_train,benign_train],ignore_index=True)
test_df = pd.concat([ransom_test,benign_test],ignore_index=True)


In [ ]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [ ]:
X_train = train_df.drop('class',axis=1)
X_test = test_df.drop('class',axis=1)
Y_train = train_df['class']
Y_test = test_df['class']


## Implementing Models

 ##   Gaussian Naive Bayes

In [ ]:
gaussian=GaussianNB(var_smoothing=0.001)
gaussian.fit(X_train,Y_train)
print('score for Gaussian Naive Bayes ',gaussian.score(X_train,Y_train))
print('score for Gaussian Naive Bayes ',gaussian.score(X_test,Y_test))
y_pred = gaussian.predict(X_test)
print('Recall Score ',recall_score(Y_test,y_pred))
print('Matrix ',confusion_matrix(Y_test, y_pred))


In [ ]:
filename = 'saved_models/gnb4.sav'
pickle.dump(gaussian, open(filename, 'wb'))

# Support Vector Machine

In [ ]:
svm = LinearSVC(C=5,penalty='l2',dual=True )
svm.fit(X_train,Y_train)
print(svm.score(X_train,Y_train))
print(svm.score(X_test,Y_test))
y_pred = svm.predict(X_test)
print("Accuracy:",accuracy_score(Y_test, y_pred))
print("Recall:",recall_score(Y_test, y_pred))
print("Matrix:",confusion_matrix(Y_test, y_pred))
print('\n')

In [ ]:
filename = 'saved_models/svm4.sav'
pickle.dump(svm, open(filename, 'wb'))

# Logistic Regression

In [ ]:
clf=LogisticRegression(solver='liblinear',C=5,penalty='l1',max_iter=1000)
clf.fit(X_train,Y_train)
pred=clf.predict(X_test)
print(clf.score(X_train,Y_train))
print('accuracy_score for LogisticRegression ',accuracy_score(Y_test,pred))
print('recall_score for LogisticRegression ',recall_score(Y_test,pred))
print(confusion_matrix(Y_test,pred))

In [ ]:
filename = 'saved_models/log4.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
!ls saved_models

# Decision Tree

In [ ]:
tree = DecisionTreeClassifier(splitter='best',criterion='entropy',max_features='auto',max_depth=8,min_samples_split=5,min_samples_leaf=20)
tree.fit(X_train, Y_train)
print("Accuracy on train set: {:.3f}".format(tree.score(X_train, Y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, Y_test)))
#print("Accuracy on val set: {:.3f}".format(tree.score(X_val, Y_val)))
p1=tree.predict(X_test)
print(confusion_matrix(Y_test,p1))
print(recall_score(Y_test,p1))

In [ ]:
filename = 'saved_models/dt4.sav'
pickle.dump(tree, open(filename, 'wb'))

In [ ]:
!ls saved_models

# RandomForest

In [ ]:
rf = RandomForestClassifier(criterion='entropy',n_estimators=1000,n_jobs=-1,max_features='auto',max_depth=7,min_samples_split=7,min_samples_leaf=6)
rf.fit(X_train,Y_train)
#print(rf.score(X_train,Y_train))
#print(rf.score(X_test,Y_test))
print(x)
y_pred = rf.predict(X_test)
print("Accuracy:",accuracy_score(Y_test, y_pred))
print("Recall:",recall_score(Y_test, y_pred))
    
print("Matrix:",confusion_matrix(Y_test, y_pred))


In [ ]:
filename = 'saved_models/rf4.sav'
pickle.dump(rf, open(filename, 'wb'))

In [ ]:
!ls saved_models

# GradientBoosting Classifier

In [ ]:
gb = GradientBoostingClassifier(loss='deviance',max_features='auto',max_depth=10,n_estimators=1000,min_samples_leaf=6,min_samples_split=4)
gb.fit(X_train, Y_train)
p=gb.predict(X_test)
print("Accuracy on test set: {:.3f}".format(gb.score(X_test, Y_test)))
print("recall on test set: {:.3f}".format(recall_score(Y_test, p)))
print(confusion_matrix(Y_test,p))

In [ ]:
filename = 'saved_models/gb4.sav'
pickle.dump(gb, open(filename, 'wb'))

In [ ]:
!ls saved_models

KNN



In [ ]:
model =KNeighborsClassifier(n_neighbors=10,weights='distance',algorithm='ball_tree',p=3) 
model.fit(X_train,Y_train)

y_pred = model.predict(X_test)
print("Accuracy:",accuracy_score(Y_test, y_pred))
print("Recall:",recall_score(Y_test, y_pred))
print(confusion_matrix(Y_test,y_pred))

In [ ]:
filename = 'saved_models/knn4.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
!ls saved_models